In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as spla
import scipy.signal as signal
import IPython.display as ipd
import soundfile as sf
import ipywidgets as ipw
import scipy.io.wavfile as wavfile
from stft_helpers import stft_window, stft

%matplotlib inline

In [2]:
def quantize(x, step, midrise=False):
    if midrise:
        y = np.floor(x / step) * step + step / 2
    else:
        y = np.floor(x / step + 0.5) * step
    return y

def dither_rpdf(step_size, n):
    return (np.random.random(n) - 0.5) * step_size

def dither_tpdf(step_size, n):
    return dither_rpdf(step_size, n) + dither_rpdf(step_size, n)

def quantize_ns(x, dither, b_h, a_h, step, midrise=False):
    a_h = np.append(a_h, 0) if len(a_h) == 1 else a_h
    y = np.zeros(x.shape)
    x_i = np.zeros(len(b_h))
    y_i = np.zeros(len(a_h)-1)
    for i, s in enumerate(x):
        h_o = (np.dot(x_i, b_h) - np.dot(y_i, a_h[1:])) / a_h[0]
        s_d = s - h_o
        y[i] = quantize(s_d + dither[i], step, midrise)
        e = y[i] - s_d
        x_i = np.append(e, x_i[0:-1])
        y_i = np.append(h_o, y_i[0:-1])
    return y

# Quantization: from simple division to dither and noise shaping

## Tsividis' paradox II

## TAM, December 16th, 2020

### Mihailo Kolundzija

## Quantization

### When and where
- A/D conversion, usually following sampling: from infinite to finite precision
- Requantization in digital domain: conversion between digital representations

### Puzzle

In [3]:
n = 10
x = np.random.randn(n)
y = np.sort(x)
def compare_sum(change):
    print(np.sum(x) == np.sum(y))

run_button = ipw.Button(description='sum(x) == sum(y) ?')
run_button.on_click(compare_sum)
display(run_button)

Button(description='sum(x) == sum(y) ?', style=ButtonStyle())

## Quantization

### Notation
- Quantized signal $x$ with values in range $[s_{\it min},s_{\it max}]$
- $[s_{\it min},s_{\it max}]$ divided into $N$ non-overlapping subintervals
- $N$ is oftentimes a power of 2 ($N=2^b$), the quantizer characterized as $b$-bit
- Quantizer maps input to the index (value between $0$ and $N-1$) of the subinterval that contains the input
- Instead of the index, the quantizer is usually considered to output the center of the corresponding interval

## Uniform quantizer
- The most common quantizer type: quantization subintervals are of equal size $\Delta$
- $\Delta$ commonly called **quantization step size** or **1 LSB**

### Uniform quantizer types
1. **Mid-tread quantizer** or **quantizer with a deadzone**
$$ Q(x) = \left\lfloor \frac{x + 0.5}{\Delta} \right\rfloor $$
2. **Mid-rise quantizer**
$$ Q(x) = \left\lfloor \frac{x}{\Delta} \right\rfloor + \frac{\Delta}{2} $$

<center>
<img src="figures/quantizer_input_output.png" width=500>
</center>

## Conventional quantization theory
- Quantization is a lossy operation
- **Quantization error** $e = y - x$

### Main assumption
- $e$ at any time instant a **uniformly distributed random variable** over $[-\Delta/2,\Delta/2)$
- $e$ viewed as a time-domain signal is a **white noise process** with zero mean and variance $\Delta^2/12$

### Signal-to-Noise Ratio (SNR)
- Expressed in reference to a peak-to-peak sinusoidal signal with amplitude $(s_{\it max}-s_{\it min})/2$
$$
\text{SNR} = 6.02 b + 1.76 ~\text{dB}
$$

- Every bit added to the representation of a signal increases the SNR by 6 dB

## Quantization error as distortion

- The assumption regarding $e$ are correct for a family of inputs with very restrictive statistical properties
- There is nothing stochastic about quantizer's operation and $e$ is deterministically related to $x$
- $e$ is unlikely to look like noise, especially if the input signal is simple and/or structured
- The correlation between $x$ and $e$ becomes apparent as one decreases the quantizer's resolution 
- Upon listening, quantization error $e$ of a low-resolution quantizer best described as *distortion*
- With audiovisuals, distortion appears much more detrimental to the signal than would be the noise of equal power

In [4]:
# The influence of the number of quantization steps on the quantization error.

# Sampling frequency in Hz.
fs = 44100
# Signal duration in seconds.
duration = 5

# Number of samples for which we plot the time-domain signals.
nt = 600
# Regularization constant to avoid taking a log of zero.
mu = 1e-6

# Define the widgets to control the demo parameters.
win_type_dd_1 = ipw.Dropdown(options=['rect', 'hanning', 'hamming'],
                           value='hanning', description='Window type',
                           disabled=False)

fft_size_dd_1 = ipw.Dropdown(options=[256, 512, 1024, 2048, 4096, 8192, 16384],
                           value=4096, description='FFT size',
                           disabled=False)

midrise_dd_1 = ipw.Dropdown(options=[('midrise', True), ('mid-tread', False)],
                          value=True, description='Quantizer type', 
                          disabled=False)

a_db_fs_1 = ipw.FloatSlider(
    min=-96, max=0, step=-3, value=-6, description='$a$ [dB]')
f_fs_1 = ipw.FloatSlider(
    min=100, max=5000, step=10, value=440, description='$f_0$')
nbits_is_1 = ipw.IntSlider(
    min=1, max=24, step=1, value=10, description='$n_{\it bits}$')

demo_out_1 = ipw.Output()

default_layout = ipw.Layout(
    border='solid 1px black',
    margin='0px 10px 10px 0px',
    padding='5px 5px 5px 5px')

def update_no_dither(a, f, nbits, midrise=False, 
                     win_type='rect', fft_size=4096):
    t = np.arange(0, duration, 1/fs)
    x = a * np.sin(2 * np.pi * f * t + 1)
    step_size = 2 / 2**nbits
    
    # Prepare the windowing function.
    win = stft_window(fft_size, fft_size, win_type=win_type)
    # Work with a 50% overlap.
    hop_size = fft_size // 2
    
    y = quantize(x, step_size, midrise)
    e = x - y
    
    # Create RPDF dither to serve as the ideal noise power reference.
    d_rpdf = dither_rpdf(step_size, len(t))
    
    # Compute the spectra of y and e by averaging the STFT magnitude.
    _, f_n, Y_s = stft(y, win, hop_size)
    _, _, E_s = stft(e, win, hop_size)
    _, _, R_s = stft(d_rpdf, win, hop_size)
    Y = np.mean(np.abs(Y_s), axis=1)
    E = np.mean(np.abs(E_s), axis=1)
    R = np.mean(np.abs(R_s), axis=1)
    # Convert normalized to absolute frequencies.
    freqs = f_n * fs
    
    e_mean = np.mean(e)
    
    e_pow_theory = step_size**2 / 12
    e_pow = np.mean(e**2)
    x_pow = np.mean(x**2)

    snr_theory = 20 * np.log10(2) * nbits + 10 * np.log10(1.5) + 20 * np.log10(a)
    snr = 10 * np.log10(x_pow / e_pow) 
    
    # Extreme value for plotted vertical lines giving harmonics.
    Y_max = 20 * np.log10(np.max(np.abs(Y)))
    Y_min = 20 * np.log10(np.min(np.abs(Y)) + mu)
    E_max = 20 * np.log10(np.max(np.abs(E)))
    E_min = 20 * np.log10(np.min(np.abs(E)) + mu)
    R_max = 20 * np.log10(np.max(np.abs(R)))
    R_min = 20 * np.log10(np.min(np.abs(R)) + mu)
    # Since we're plotting the theoretical noise PSD, account for it.
    Y_max = np.maximum(Y_max, R_max)
    Y_min = np.minimum(Y_min, R_min)
    E_max = np.maximum(E_max, R_max)
    E_min = np.minimum(E_min, R_min)
    
    # Harmonics of the sinusoidal signal to show on plot.
    harmonics = np.arange(f, fs/2, f)
    
    out_text_0 = ipw.Output()
    out_text_0.layout = default_layout
    out_text_1 = ipw.Output()
    out_text_1.layout = default_layout
    
    with out_text_0:
        print('Quantization step size:            {:.4E}'.format(step_size))
        print('Quantization error power (theory): {:.4E}'.format(e_pow_theory))
        print('Quantization error power:          {:.4E}'.format(e_pow))
    with out_text_1:
        print('SNR (theory): {:.4f} dB'.format(snr_theory))
        print('SNR:          {:.4f} dB'.format(snr))
        
    audio_out_0 = ipw.Output()
    audio_out_1 = ipw.Output()
    audio_out_2 = ipw.Output()
    
    with audio_out_0:
        print('Original')
        display(ipd.Audio(data=np.clip(x, -1, 1), rate=fs, normalize=False))
    with audio_out_1:
        print('Quantized')
        display(ipd.Audio(data=np.clip(y, -1, 1), rate=fs, normalize=False))
    with audio_out_2:
        print('Quantization error')
        display(ipd.Audio(data=np.clip(e, -1, 1), rate=fs, normalize=False))
    
    text_group = ipw.HBox([out_text_0, out_text_1])
    audio_group = ipw.HBox([audio_out_0, audio_out_1, audio_out_2])
    audio_group.layout = default_layout
    
    demo_out_1.clear_output()
    
    figure_out = ipw.Output()
    figure_out.layout = default_layout
    with figure_out:
        plt.figure(figsize=(15, 6))

        plt.subplot(2, 2, 1)
        plt.plot(t[:nt], y[:nt], color='b')
        plt.xlabel('t [s]')
        plt.title('Quantizer output $y$')
        plt.grid()

        plt.subplot(2, 2, 3)
        plt.plot(t[:nt], e[:nt], color='b')
        plt.xlabel('t [s]')
        plt.title('Quantization error $e$')
        plt.grid()

        plt.subplot(2, 2, 2)
        plt.vlines(harmonics, Y_min, Y_max, label='$f_0$ harmonics', 
                   colors='yellow', linestyles='dashed')
        plt.plot(freqs, 20 * np.log10(np.abs(Y)+mu), 
                 color='b', label='$Y(f)$')
        plt.plot(freqs, 20 * np.log10(np.abs(R)+mu), 
                 color='red', label='$Ref. noise PSD$')
        plt.xlabel('f [Hz]')
        plt.title('Spectrum $Y(f)$ of the quantized signal')
        plt.legend()
        plt.grid()

        plt.subplot(2, 2, 4)
        plt.vlines(harmonics, E_min, E_max, label='$f_0$ harmonics', 
                   color='yellow', linestyles='dashed')
        plt.plot(freqs, 20 * np.log10(np.abs(E)+mu), 
                 color='b', label='$E(f)$')
        plt.plot(freqs, 20 * np.log10(np.abs(R)+mu), 
                 color='red', label='$Ref. noise PSD$')
        plt.xlabel('f [Hz]')
        plt.title('Spectrum $E(f)$ of the quantization error')
        plt.legend()
        plt.grid()

        plt.tight_layout()

        plt.show()
    
    with demo_out_1:
        display(text_group)
        display(audio_group)
        display(figure_out)
        
def click_no_dither(change):
    update_no_dither(a=10**(a_db_fs_1.value/20),
                     f=f_fs_1.value,
                     nbits=nbits_is_1.value,
                     win_type=win_type_dd_1.value, 
                     fft_size=fft_size_dd_1.value,
                     midrise=midrise_dd_1.value)

# Group widgets to have the desired looks.
sliders_1 = ipw.HBox([a_db_fs_1, f_fs_1, nbits_is_1])
sliders_1.layout = default_layout
dropdowns_1 = ipw.HBox([midrise_dd_1, win_type_dd_1, fft_size_dd_1])
dropdowns_1.layout = default_layout
controls_1 = ipw.VBox([sliders_1, dropdowns_1])
controls_1.layout = default_layout
run_button_1 = ipw.Button(description='Run')
run_button_1.on_click(click_no_dither)

### Quantization distortion demo

In [5]:
display(ipw.VBox([controls_1, run_button_1, demo_out_1]))
click_no_dither(None)

## Dithered quantization
- Having $e$ appear as noise is more desirable than signal-dependent distortion 
- Adding noise (analog or digital) called **dither** prior to quantization turns quantization distortion into noise 
- The price can be a small decrease in SNR

## Types of dithered quantizers
1. Subtractively dithered
2. Non-subtractively dithered

<center>
<img src="figures/quantizer_dither.png" width=400>
</center>

## Subtractively dithered quantizers

- Add dither $\nu$ prior to quantization, but then the same dither $\nu$ gets subtracted at the receiver
- Receiver needs to be in sync with the sender in order to be able to reproduce the same dither

### Main result

1. One can design simple dither signals which *for any input signal* turn the quantization error $$e=Q(x+\nu) - \nu - x$$ into **white noise independent from the input**
2. An iid dither uniformly distributed (with RPDF) inside  $[-\Delta/2,\Delta/2)$ guarantees that
3. With such dither, quantization error's power equals $\Delta^2/12$, which is precisely what happens in the idealized, conventional (or high-resolution) quantization model

## Non-subtractively dithered quantizers

- Non-subtractively dithered quantizers are more practical, but practicality comes with a price

### Main result
1. One cannot make quantization error $$ e = Q(x + \nu) - x $$ statistically independent from the input
2. By appropriately choosing dither's pdf, the moments of the quantization error $e$ can be made independent from the input

## Non-subtractively dithered quantizers

### RPDF dither
- If dither $\nu$ has RPDF in $[-\Delta/2,\Delta/2)$, the error $e$ is zero-mean irrespective of the input
- Power of $e$ is not independent of $x$ and this phenomenon is called **noise modulation**

<center>
<img src="figures/dither_pdf.png" width=500>
</center>

### TPDF dither
- Dither with TPDF in $[-\Delta,\Delta)$ makes $e$ zero-mean and its power independent from $x$ and equal to $\Delta^2/4$
- $e$ is not iid, but **its spectrum is white**

In [6]:
# What happens when we add the subtractive dither into the picture.

# Sampling frequency in Hz.
fs = 44100
# Signal duration in seconds.
duration = 4

# Number of samples for which we plot the time-domain signals.
nt = 600
# Regularization constant to avoid taking a log of zero.
mu = 1e-6

# Define the widgets to control the demo parameters.
dither_opt_dd_2 = ipw.Dropdown(
    options=['no', 'subtractive', 'non-subtractive'],
    value='no', description='Dither: ', disabled=False)

dither_type_dd_2 = ipw.Dropdown(
    options=['RPDF', 'TPDF'],
    value='RPDF', description='Dither type: ',
    disabled=False)

midrise_dd_2 = ipw.Dropdown(
    options=[('midrise', True), ('mid-tread', False)],
    value=True, description='Quantizer type',
    disabled=False)

win_type_dd_2 = ipw.Dropdown(
    options=['rect', 'hanning', 'hamming'],
    value='hanning', description='Window type',
    disabled=False)

fft_size_dd_2 = ipw.Dropdown(
    options=[256, 512, 1024, 2048, 4096, 8192, 16384],
    value=4096, description='FFT size',
    disabled=False)

a_fs_2 = ipw.FloatSlider(
    min=0, max=1, step=0.001, value=0.5, description='$a$')
f_fs_2 = ipw.FloatSlider(
    min=100, max=5000, step=10, value=440, description='$f_0$')
nbits_is_2 = ipw.IntSlider(
    min=1, max=24, step=1, value=10, description='$n_{\it bits}$')

demo_out_2 = ipw.Output()

default_layout = ipw.Layout(
    border='solid 1px black',
    margin='0px 10px 10px 0px',
    padding='5px 5px 5px 5px')

def update_dither(
    a, f, nbits, midrise=False, 
    dither_opt='no', dither_type='rpdf',
    win_type='rect', fft_size=4096):
    
    t = np.arange(0, duration, 1/fs)
    x = a * np.sin(2 * np.pi * f * t + 1)
    step_size = 2 / 2**nbits
    
    # Prepare the windowing function.
    win = stft_window(fft_size, fft_size, win_type=win_type)
    # Work with a 50% overlap.
    hop_size = fft_size // 2
    
    snr_reduction = 0
    e_pow_mul_increase = 1
    
    if dither_type.lower() == 'rpdf':
        d = dither_rpdf(step_size, len(t))
        snr_reduction = 10 * np.log10(2)
        e_pow_mul_increase = 2
    elif dither_type.lower() == 'tpdf':
        d = dither_tpdf(step_size, len(t))
        snr_reduction = 10 * np.log10(3)
        e_pow_mul_increase = 3
    else:
        raise ValueError('Unknown dither type')
    
    snr_theory = 20 * np.log10(2) * nbits + 10 * np.log10(1.5)
    # Correct for signal's amplitude.
    snr_theory += 20 * np.log10(a)
    e_pow_theory = step_size**2 / 12
    
    if dither_opt.lower() == 'no':
        y = quantize(x, step_size, midrise)
    elif dither_opt.lower() == 'subtractive':
        e_pow_mul_increase = 1
        y = quantize(x + d, step_size, midrise) - d
    elif dither_opt.lower() == 'non-subtractive':
        y = quantize(x + d, step_size, midrise)
        snr_theory -= snr_reduction
        e_pow_theory *= e_pow_mul_increase
    else:
        raise ValueError('Unknown dithering subtraction scheme')
    
    # Create RPDF dither to serve as the ideal noise power reference.
    d_rpdf = dither_rpdf(step_size, len(t))
        
    e = y - x
    
    # Compute the spectra of the quantizer's output and error.
    _, f_n, Y_s = stft(y, win, hop_size)
    _, _, E_s = stft(e, win, hop_size)
    _, _, D_s = stft(d, win, hop_size)
    _, _, R_s = stft(d_rpdf, win, hop_size)
    Y = np.mean(np.abs(Y_s), axis=1)
    E = np.mean(np.abs(E_s), axis=1)
    D = np.mean(np.abs(D_s), axis=1)
    R = np.mean(np.abs(R_s), axis=1)
    # Convert normalized to absolute frequencies.
    freqs = f_n * fs
    
    # Compute the stats.
    e_mean = np.mean(e)
    e_pow = np.mean(e**2)
    d_pow = np.mean(d**2)
    x_pow = np.mean(x**2)
    r_pow = np.mean(d_rpdf**2)

    snr = 10 * np.log10(x_pow / e_pow)
    
    # Extreme value for plotted vertical lines showing harmonics.
    Y_max = 20 * np.log10(np.max(np.abs(Y)))
    Y_min = 20 * np.log10(np.min(np.abs(Y)) + mu)
    E_max = 20 * np.log10(np.max(np.abs(E)))
    E_min = 20 * np.log10(np.min(np.abs(E)) + mu)
    R_max = 20 * np.log10(np.max(np.abs(R)))
    R_min = 20 * np.log10(np.min(np.abs(R)) + mu)
    # Since we're plotting the theoretical noise PSD, account for them.
    Y_max = np.maximum(Y_max, R_max)
    Y_min = np.minimum(Y_min, R_min)
    E_max = np.maximum(E_max, R_max)
    E_min = np.minimum(E_min, R_min)
    
    # Harmonics of the sinusoid's frequency to show on plots.
    harmonics = np.arange(f, fs/2, f)
    
    out_text_0 = ipw.Output()
    out_text_0.layout = default_layout
    out_text_1 = ipw.Output()
    out_text_1.layout = default_layout
    
    with out_text_0:
        print('Quantization step size: {:.4E}'.format(step_size))
        print('Noise power (theory):   {:.4E}'.format(e_pow_theory))
        print('Noise power:            {:.4E}'.format(e_pow))
    with out_text_1:
        print('Dither power: {:.4E}'.format(d_pow))
        print('SNR (theory): {:.4f} dB'.format(snr_theory))
        print('SNR:          {:.4f} dB'.format(snr))
        
    audio_out_0 = ipw.Output()
    audio_out_1 = ipw.Output()
    audio_out_2 = ipw.Output()
    
    with audio_out_0:
        print('Original')
        display(ipd.Audio(data=np.clip(x, -1, 1), 
                              rate=fs, normalize=False))
    with audio_out_1:
        print('Quantized')
        display(ipd.Audio(data=np.clip(y, -1, 1), 
                              rate=fs, normalize=False))
    with audio_out_2:
        print('Quantization error')
        display(ipd.Audio(data=np.clip(e, -1, 1), rate=fs, normalize=False))
    
    text_group = ipw.HBox([out_text_0, out_text_1])
    audio_group = ipw.HBox([audio_out_0, audio_out_1, audio_out_2])
    audio_group.layout = default_layout
    
    demo_out_2.clear_output()
    
    figure_out = ipw.Output()
    figure_out.layout = default_layout

    with figure_out:
        plt.figure(figsize=(15, 6))

        plt.subplot(2, 2, 1)
        plt.plot(t[:nt], y[:nt], color='b')
        plt.ylim([-1.1, 1.1])
        plt.title('Quantizer\'s output')
        plt.ylabel('$y(t)$')
        plt.grid()

        plt.subplot(2, 2, 3)
        plt.plot(t[:nt], e[:nt], color='b')
        plt.xlabel('$t$ [s]')
        plt.ylabel('$e(t)$')
        plt.title('Quantization error')
        plt.grid()

        plt.subplot(2, 2, 2)
        plt.vlines(harmonics, Y_min, Y_max, label='$f_0$ harmonics', 
                   colors='yellow', linestyles='dashed')
        plt.plot(freqs, 20 * np.log10(np.abs(Y)+mu), 
                 color='blue', label='$Y(f)$')
        plt.plot(freqs, 20 * np.log10(np.abs(R)+mu), 
                 color='red', label='$Ref. noise PSD$')
        plt.ylabel('$Y(f)$ [dB]')
        plt.title('Quantized signal\'s spectrum')
        plt.legend()
        plt.grid()

        plt.subplot(2, 2, 4)
        plt.vlines(harmonics, E_min, E_max, label='$f_0$ harmonics', 
                   color='yellow', linestyles='dashed')
        plt.plot(freqs, 20 * np.log10(np.abs(E)+mu), 
                 color='blue', label='$E(f)$')
        plt.plot(freqs, 20 * np.log10(np.abs(R)+mu), 
                 color='red', label='$Ref. noise PSD$')
        plt.ylabel('$E(f)$ [dB]')
        plt.xlabel('$f$ [Hz]')
        plt.title('Quantization error\'s spectrum')
        plt.legend()
        plt.grid()

        plt.tight_layout()

        plt.show()
        
    with demo_out_2:
        display(text_group)
        display(audio_group)
        display(figure_out)

def click_dither(change):
    update_dither(
        a=a_fs_2.value, f=f_fs_2.value,
        nbits=nbits_is_2.value,
        midrise=midrise_dd_2.value,
        dither_opt=dither_opt_dd_2.value, 
        dither_type=dither_type_dd_2.value,
        win_type=win_type_dd_2.value,
        fft_size=fft_size_dd_2.value)

# Group widgets to have the desired looks.
sliders_2 = ipw.HBox([a_fs_2, f_fs_2, nbits_is_2])
sliders_2.layout = default_layout
dropdowns0_2 = ipw.HBox([midrise_dd_2, dither_opt_dd_2, dither_type_dd_2])
dropdowns1_2 = ipw.HBox([win_type_dd_2, fft_size_dd_2])
dropdowns_2 = ipw.VBox([dropdowns0_2, dropdowns1_2])
dropdowns_2.layout = default_layout
controls_2 = ipw.VBox([sliders_2, dropdowns_2])
controls_2.layout = default_layout
run_button_2 = ipw.Button(description='Run')
run_button_2.on_click(click_dither)

### Dithered quantization demo

In [7]:
display(ipw.VBox([controls_2, run_button_2, demo_out_2]))
click_dither(None)

In [8]:
# Parameters of the signal.
fs = 44100
duration = 4
amplitude = 0.5

# STFT parameters.
fft_size = 4096
win_type = 'hanning'
hop_size = fft_size // 2

# Prepare the windowing function.
win_3 = stft_window(fft_size, fft_size, win_type=win_type)

f_fs_3 = ipw.FloatSlider(
    min=100, max=5000, step=10, value=440, description='$f_0$')

nbits_is_3 = ipw.IntSlider(
    min=1, max=24, step=1, value=4, 
    description='$n_{\it bits}$', continuous_update=False)

midrise_cb_3 = ipw.Checkbox(description='midrise', value=True)

demo_out_3 = ipw.Output()

# Prepare the source signal by applying a ramp to it.
t = np.arange(0, duration, 1/fs)

def update_noise_modulation(f=440, nbits=4, midrise=True, win=win_3):
    x = amplitude * np.sin(2 * np.pi * f * t + 1)
    x = x * np.linspace(0, 1, len(t))
    n = len(t)
    
    nsteps = 2**nbits
    # We assume that the signal amplitude is in interval [-1, 1]
    step_size = 2 / nsteps

    t_stft, f_stft, x_stft = stft(x, win, hop_size)
    x_psd = np.abs(x_stft)**2
    x_pow = np.sum(np.abs(x_psd), axis=0)

    # Prepare the dither (RPDF and TPDF).
    d_rpdf = dither_rpdf(step_size, n)
    d_tpdf = dither_tpdf(step_size, n)

    # Perform quantization -- without and with dither.
    y_ramp_nq = quantize(x, step_size, midrise)
    y_ramp_rpdf = quantize(x + d_rpdf, step_size, midrise)
    y_ramp_tpdf = quantize(x + d_tpdf, step_size, midrise)

    # Compute quantization error.
    e_ramp_nq = y_ramp_nq - x
    e_ramp_rpdf = y_ramp_rpdf - x
    e_ramp_tpdf = y_ramp_tpdf - x

    # Compute the exponentially-averaged PSD in the STFT domain.
    _, _, e_ramp_nq_stft = stft(e_ramp_nq, win, hop_size)
    e_ramp_nq_psd = np.abs(e_ramp_nq_stft)**2
    _, _, e_ramp_rpdf_stft = stft(e_ramp_rpdf, win, hop_size)
    e_ramp_rpdf_psd = np.abs(e_ramp_rpdf_stft)**2
    _, _, e_ramp_tpdf_stft = stft(e_ramp_tpdf, win, hop_size)
    e_ramp_tpdf_psd = np.abs(e_ramp_tpdf_stft)**2

    # Compute the change of power in time from the STFT represenation.
    e_ramp_nq_pow = np.sum(np.abs(e_ramp_nq_psd), axis=0)
    e_ramp_rpdf_pow = np.sum(np.abs(e_ramp_rpdf_psd), axis=0)
    e_ramp_tpdf_pow = np.sum(np.abs(e_ramp_tpdf_psd), axis=0)

    # Compute the average PSD over time.
    e_ramp_nq_spec = np.sum(np.abs(e_ramp_nq_psd), axis=1)
    e_ramp_rpdf_spec = np.sum(np.abs(e_ramp_rpdf_psd), axis=1)
    e_ramp_tpdf_spec = np.sum(np.abs(e_ramp_tpdf_psd), axis=1)

    audio_out_0 = ipw.Output()
    with audio_out_0:
        label = 'Original'
        display(ipw.Label(label))
        display(ipd.Audio(data=np.clip(x, -1, 1), 
                          rate=fs, normalize=False))
    audio_out_1 = ipw.Output()
    with audio_out_1:
        label = 'Quantized w/ RPDF dither'
        display(ipw.Label(label))
        display(ipd.Audio(data=np.clip(y_ramp_rpdf, -1, 1), 
                              rate=fs, normalize=False))
    audio_out_2 = ipw.Output()
    with audio_out_2:
        label = 'Quantized w/ TPDF dither'
        display(ipw.Label(label))
        display(ipd.Audio(data=np.clip(y_ramp_tpdf, -1, 1), 
                              rate=fs, normalize=False))
    audio_out_3 = ipw.Output()
    with audio_out_3:
        label = 'Quantization error w/ RPDF dither'
        display(ipw.Label(label))
        display(ipd.Audio(data=np.clip(e_ramp_rpdf, -1, 1), 
                              rate=fs, normalize=False))
    audio_out_4 = ipw.Output()
    with audio_out_4:
        label = 'Quantization error w/ TPDF dither'
        display(ipw.Label(label))
        display(ipd.Audio(data=np.clip(e_ramp_tpdf, -1, 1), 
                              rate=fs, normalize=False))

    audio_group_1 = ipw.VBox([audio_out_1, audio_out_3])
    audio_group_2 = ipw.VBox([audio_out_2, audio_out_4])

    audio_pairs = ipw.HBox([audio_out_0, audio_group_1, audio_group_2])
    audio_pairs.layout = default_layout

    figure_out = ipw.Output()
    figure_out.layout = default_layout

    with figure_out:
        plt.figure(figsize=(8, 5))

        # Get maximum and minimum powers for y-axis limits.
        e_pow_max = np.max(np.append(e_ramp_rpdf_pow, e_ramp_tpdf_pow))
        e_pow_min = np.min(np.append(e_ramp_rpdf_pow, e_ramp_tpdf_pow))

        plt.subplot(3, 1, 1)
        plt.plot(t_stft / fs, 10 * np.log10(x_pow), color='blue')
        plt.ylabel('$P(t)$ [dB]')
        plt.title('Signal power over time')
        plt.grid()

        plt.subplot(3, 1, 2)
        plt.plot(t_stft / fs, 10 * np.log10(e_ramp_rpdf_pow), color='blue')
        plt.ylabel('$P(t)$ [dB]')
        plt.title('Quantization error power over time with RPDF dither')
        plt.ylim(10*np.log10(e_pow_min)-1, 10*np.log10(e_pow_max)+1)
        plt.grid()

        plt.subplot(3, 1, 3)
        plt.plot(t_stft / fs, 10 * np.log10(e_ramp_tpdf_pow), color='blue')
        plt.xlabel('$t$ [s]')
        plt.ylabel('$P(t)$ [dB]')
        plt.title('Quantization error power over time with TPDF dither')
        plt.ylim(10*np.log10(e_pow_min)-1, 10*np.log10(e_pow_max)+1)
        plt.grid()

        plt.tight_layout()
        plt.show()
    
    demo_out_3.clear_output()
    with demo_out_3:
        display(ipw.VBox([audio_pairs, figure_out]))
        
def click_noise_modulation(change):
    update_noise_modulation(
        f=f_fs_3.value, nbits=nbits_is_3.value, midrise=midrise_cb_3.value, win=win_3)

controls_3 = ipw.HBox([f_fs_3, nbits_is_3, midrise_cb_3])
controls_3.layout = default_layout
run_button_3 = ipw.Button(description='Run')
run_button_3.on_click(click_noise_modulation)

### RPDF vs. TPDF dither: noise modulation

In [9]:
display(ipw.VBox([controls_3, run_button_3, demo_out_3]))
click_noise_modulation(None)

## Noise shaping

- Dithered quantizers are able to make the quantization error behave similarly to white noise
- Improvements, at least in a perceptual sense, can be obtained by processing the quantization noise
- The principal idea is to feed back the quantization error through a filter and subtract it from the input
<center>
<img src="figures/quantizer_noise_shape.png" width=400>
</center>

## Noise shaping

- The output signal $y$ consists of the input $x$ and the quantization error $e$ filtered by $1-h$ $$ y = x + e * (1-h) $$
- Filter $h$ is a causal filter with $h_0=0$, namely it has to have a delay of at least one sample
- If TPDF dither is used, the quantization error resembles white noise with power spectrum $$P(e^{j\omega})=\Delta^2/4$$
- The power spectrum of the quantization error with noise shaping is given by $$P(e^{j\omega})= \frac{\Delta^2}{4}|1-H(e^{j\omega})|^2$$
- The noise shaping filter design seeks $1-H(e^{j\omega})$ that makes quantization noise *least audible*

## Noise shaping: perceptual motivation

- A signal of a given power can appear differently loud at different frequencies ([equal loudness contours](https://en.wikipedia.org/wiki/Equal-loudness_contour))
- Loudness of quantization noise reduced optimally when $1-h$ inverts our sensitivity to low-level noise
<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/4/49/Lindos4.svg" width=360>
</center>

In [10]:
# The influence of the frequency and the loudness of a sinuosid.

# Sampling frequency in Hz.
fs = 44100
# Signal duration in seconds.
duration = 5
# Frequency of the reference tone (1 kHz).
f_ref = 1000

t = np.arange(0, duration, 1/fs)

a_db_fs_4 = ipw.FloatSlider(
    min=-96, max=0, step=-3, value=-40, description='$a$ [dB]')
f_fs_4 = ipw.FloatSlider(
    min=100, max=15000, step=10, value=440, description='$f_0$')
a_ref_db_fs_4 = ipw.FloatSlider(
    min=-96, max=0, step=-3, value=-40, description='$a_{\it ref}$ [dB]')

demo_out_4 = ipw.Output()

def update_sin_loudness(a, f, a_ref):
    x = a * np.sin(2 * np.pi * f * t + 1)
    x_ref = a_ref * np.sin(2 * np.pi * f_ref * t + 1)
    
    audio1_out = ipw.Output()
    with audio1_out:
        label = 'Ref. tone (f = {:d} Hz)'.format(f_ref)
        display(ipw.Label(label))
        display(ipd.Audio(data=np.clip(x_ref, -1, 1), rate=fs, normalize=False))
    audio2_out = ipw.Output()
    with audio2_out:
        label = 'Test tone (f = {:.2f} Hz)'.format(f)
        display(ipw.Label(label))
        display(ipd.Audio(data=np.clip(x, -1, 1), rate=fs, normalize=False))
        
    audio_group = ipw.HBox([audio1_out, audio2_out])
    
    demo_out_4.clear_output()
    
    figure_out = ipw.Output()
    figure_out.layout = default_layout
    
    with demo_out_4:
        display(audio_group)
        
def click_sin_loudness(change):
    update_sin_loudness(a=10**(a_db_fs_4.value/20), f=f_fs_4.value,
        a_ref=10**(a_ref_db_fs_4.value/20))

# Group widgets to have the desired looks.
controls_4 = ipw.HBox([a_db_fs_4, f_fs_4, a_ref_db_fs_4])
controls_4.layout = default_layout
run_button_4 = ipw.Button(description='Run')
run_button_4.on_click(click_sin_loudness)

In [11]:
display(ipw.VBox([controls_4, run_button_4, demo_out_4]))
click_sin_loudness(None)

## Noise shaping: key practical facts

- The spectrum of the quantization error with TPDF dither and noise shaping filter is $$ P(e^{j\omega}) = \frac{\Delta^2}{4} |1-H(e^{j\omega})|^2 $$
- Filter $H(e^{j\omega})$ is computed to minimize the weighted noise power $$ \frac{1}{2\pi} \int_{-\pi}^{\pi} P(e^{j\omega})\, W(e^{j\omega})\, \mathrm{d}\omega = \frac{1}{2\pi} \int_{-\pi}^{\pi} \frac{\Delta^2}{4} |1-H(e^{j\omega})|^2\, W(e^{j\omega})\, \mathrm{d}\omega$$
- $W(e^{j\omega})$ is a non-negative function of frequency that models human auditory system's sensitivity at low noise levels
- $W(e^{j\omega})$ usually derived from the $15$-phon equal loudness contour

## Noise shaping: key results
1. For a noise shaping filter of the form $1-H(e^{j\omega})$, where $H(e^{j\omega})$ has no constant terms, we have $$ \int_{-\pi}^{\pi} \log\left|1-H(e^{j\omega})\right|^2 \mathrm{d}\omega \geq 0; $$ the equality holds when $1-H(e^{j\omega})$ is [minimum-phase](https://en.wikipedia.org/wiki/Minimum_phase)
2. The optimal noise shaping filter $1-H(e^{j\omega})$ that minimizes the weighted power $$ \frac{1}{2\pi} \int_{-\pi}^{\pi} P(e^{j\omega})\, W(e^{j\omega})\, \mathrm{d}\omega $$ is minimum-phase and satisfies $$ |1-H(e^{j\omega})|^2 = \frac{w}{W(e^{j\omega})} $$
3. Given the result in point (1), the constant $w$ easily obtained and has the value $$ w = \exp\left({\frac{1}{2\pi} \int_{-\pi}^{\pi} W(e^{j\omega}) \mathrm{d}\omega}\right) $$

## Noise shaping: key results

- The optimal noise shaping filter, the areas under and above the unit gain (or 0 dB) must be *equal on the logarithmic scale*
- Attenuating the quantization noise at some frequencies will inevitably amplify it at others
- The area above the unit gain *will always be bigger on the linear scale* ([Jensen's inequality](https://en.wikipedia.org/wiki/Jensen%27s_inequality))
- The *total quantization noise power with a noise shaping filter will always increase compared to no noise shaping*

In [12]:
# Sampling frequency in Hz.
fs = 44100
# Signal duration in seconds.
duration = 4

# Number of samples for which we plot the time-domain signals.
nt = 600
# Regularization constant to avoid taking a log of zero.
mu = 1e-6

# Define the noise shaping filter.
a_h = np.array([1])
# b_h = np.array([1])
# b_h = np.array([1.623, -0.982, 0.109])
b_h = np.array([2.412, -3.37, 3.937, -4.174, 3.353, -2.205, 1.281, -0.569, 0.0847])

# Define the widgets to control the demo parameters.
dither_type_dd_5 = ipw.Dropdown(
    options=['RPDF', 'TPDF'],
    value='TPDF', description='Dither type: ',
    disabled=False)

midrise_dd_5 = ipw.Dropdown(
    options=[('midrise', True), ('mid-tread', False)],
    value=True, description='Quantizer type', 
    disabled=False)

win_type_dd_5 = ipw.Dropdown(options=['rect', 'hanning', 'hamming'],
    value='hanning', description='Window type',
    disabled=False)

fft_size_dd_5 = ipw.Dropdown(
    options=[256, 512, 1024, 2048, 4096, 8192, 16384],
    value=4096, description='FFT size',
    disabled=False)

a_fs_5 = ipw.FloatSlider(
    min=0, max=1, step=0.001, value=0.5, description='$a$')

f_fs_5 = ipw.FloatSlider(
    min=100, max=5000, step=10, value=440, description='$f_0$')

nbits_is_5 = ipw.IntSlider(
    min=1, max=24, step=1, value=10, description='$n_{\it bits}$')

demo_out_5 = ipw.Output()

default_layout = ipw.Layout(
    border='solid 1px black',
    margin='0px 10px 10px 0px',
    padding='5px 5px 5px 5px')

def update_ns_dither(
    a, f, T, fs, nbits, midrise=False,
    dither_type='rpdf',
    win_type='rect', fft_size=4096):
    
    t = np.arange(0, T, 1/fs)
    x = a * np.sin(2 * np.pi * f * t)
    step_size = 2 / 2**nbits
    
    # Prepare the windowing function.
    win = stft_window(fft_size, fft_size, win_type=win_type)
    # Work with a 50% overlap.
    hop_size = fft_size // 2
    
    if dither_type.lower() == 'rpdf':
        d = dither_rpdf(step_size, len(t))
        e_pow_theory = step_size**2 / 6
    elif dither_type.lower() == 'tpdf':
        d = dither_tpdf(step_size, len(t))
        e_pow_theory = step_size**2 / 4
    else:
        raise ValueError('Unknown dither type')

    y_nns = quantize(x + d, step_size, midrise)
    y_ns = quantize_ns(x, d, b_h, a_h, step_size, midrise)
    e_ns = y_ns - x
    e_nns = y_nns - x

    # Compute the quantized signal's spectrum.
    _, f_n, Y_ns_s = stft(y_ns, win, hop_size)
    Y_ns = np.mean(np.abs(Y_ns_s), axis=1)
    _, _, Y_nns_s = stft(y_nns, win, hop_size)
    Y_nns = np.mean(np.abs(Y_nns_s), axis=1)
    # Compute the error spectra.
    _, _, E_ns_s = stft(e_ns, win, hop_size)
    E_ns = np.mean(np.abs(E_ns_s), axis=1)
    _, _, E_nns_s = stft(e_nns, win, hop_size)
    E_nns = np.mean(np.abs(E_nns_s), axis=1)
    
    freqs = f_n * fs
    
    # Extreme value for plotted vertical lines showing harmonics.
    Y_max = 20 * np.log10(np.max(np.abs(Y_ns)))
    Y_min = 20 * np.log10(np.min(np.abs(Y_ns)) + mu)
    E_max = 20 * np.log10(np.max(np.abs(E_ns)))
    E_min = 20 * np.log10(np.min(np.abs(E_ns)) + mu)
    # Since we're plotting the theoretical noise PSD, account for them.
    Y_max = np.maximum(Y_max, 10*np.log10(e_pow_theory*fft_size))
    Y_min = np.minimum(Y_min, 10*np.log10(e_pow_theory*fft_size))
    E_max = np.maximum(E_max, 10*np.log10(e_pow_theory*fft_size))
    E_min = np.minimum(E_min, 10*np.log10(e_pow_theory*fft_size))
    
    audio_out_0 = ipw.Output()
    audio_out_1 = ipw.Output()
    audio_out_2 = ipw.Output()
    
    with audio_out_0:
        print('Original')
        display(ipd.Audio(data=np.clip(x, -1, 1), rate=fs, normalize=False))
    with audio_out_1:
        print('Quantized w/ {:s} dither'.format(dither_type.upper()))
        display(ipd.Audio(data=np.clip(y_nns, -1, 1), rate=fs, normalize=False))
    with audio_out_2:
        print('Quantized w/ {:s} dither and noise shaping'.format(dither_type.upper()))
        display(ipd.Audio(data=np.clip(y_ns, -1, 1), rate=fs, normalize=False))
        
    audio_group = ipw.HBox([audio_out_0, audio_out_1, audio_out_2])
    audio_group.layout = default_layout
    
    demo_out_5.clear_output()
    
    figure_out = ipw.Output()
    figure_out.layout = default_layout

    with figure_out:
        plt.figure(figsize=(15, 6))

        plt.subplot(2, 2, 1)
        plt.plot(t[:nt], y_ns[:nt], color='b')
        plt.ylabel('$y(t)$')
        plt.title('Quantizer\'s output with noise shaping', fontsize=12)
        plt.ylim([-1.1, 1.1])
        plt.grid()

        plt.subplot(2, 2, 3)
        plt.plot(t[:nt], e_ns[:nt], color='b')
        plt.xlabel('$t$ [s]')
        plt.ylabel('$e(t)$')
        plt.title('Quantization error with noise shaping', fontsize=12)
        plt.grid()

        plt.subplot(2, 2, 2)
        plt.plot(freqs, 20*np.log10(np.abs(Y_ns)+mu), color='blue', label='w/ NS')
        plt.plot(freqs, 20*np.log10(np.abs(Y_nns)+mu), color='green', label='w/o NS')
        plt.ylabel('$Y(f)$ [dB]')
        plt.title('Quantized signal\'s spectrum')
        plt.legend()
        plt.grid()

        plt.subplot(2, 2, 4)
        plt.plot(freqs, 20*np.log10(np.abs(E_ns)+mu), 
                 color='blue', label='w/ NS')
        plt.plot(freqs, 20*np.log10(np.abs(E_nns)+mu),
                 color='green', label='w/o NS')
        plt.xlabel('$f$ [H]')
        plt.ylabel('$E(f)$ [dB]')
        plt.title('Quantization error\'s spectrum')
        plt.legend()
        plt.grid()

        plt.tight_layout()
        plt.show()
        
    with demo_out_5:
        display(audio_group)
        display(figure_out)
        
def click_ns_dither(change):
    update_ns_dither(
        a=a_fs_5.value, f=f_fs_5.value,
        nbits=nbits_is_5.value,
        T=duration, fs=fs,
        midrise=midrise_dd_5.value,
        dither_type=dither_type_dd_5.value,
        win_type=win_type_dd_5.value, 
        fft_size=fft_size_dd_5.value)

# Group widgets to have the desired looks.
sliders_5 = ipw.HBox([a_fs_5, f_fs_5, nbits_is_5])
sliders_5.layout = default_layout
dropdowns0_5 = ipw.HBox([midrise_dd_5, dither_type_dd_5])
dropdowns1_5 = ipw.HBox([win_type_dd_5, fft_size_dd_5])
dropdowns_5 = ipw.VBox([dropdowns0_5, dropdowns1_5])
dropdowns_5.layout = default_layout
controls_5 = ipw.VBox([sliders_5, dropdowns_5])
controls_5.layout = default_layout
run_button_5 = ipw.Button(description='Run')
run_button_5.on_click(click_ns_dither)

### Dithered quantization with noise shaping demo

In [ ]:
display(ipw.VBox([controls_5, run_button_5, demo_out_5]))
click_ns_dither(None)

## A different perspective on the Tsividis paradox

*Listening to a CD with Ravel's Bolero (known for its use of a long crescendo), Tsividis noticed atrocious distortions when a flute plays - quietly of course - at the very beginning of the piece. Given those, he posed a question on the kind of interplay between analog, digital, and the quantization process itself that could explain the unrecognizable all-but-pleasant sound of the flute*

- My explanation is very simple, which doesn't mean it's correct (Occam's razor can be only a razor, after all)
- I think the annoying distortion could be attributed to the inadequacy of (re)quantization, probably due to the lack of appropriate dithering and/or noise shaping used for mastering the CD

In [ ]:
audio_file = 'audio/Bolero.wav'

fs, data = wavfile.read(audio_file)
if issubclass(data.dtype.type, np.integer):
    data = 1.0 * data / abs(np.iinfo(data.dtype).min)

# STFT parameters.
fft_size = 2**int(round(np.log2(fs/2))) # roughly 0.5 seconds
block_overlap = 0.5
win_type = 'hanning'

hop_size = int(round(fft_size * (1 - block_overlap)))

# Prepare the windowing function.
win = stft_window(fft_size, fft_size, win_type=win_type)

# Take only the left channel.
x = data[:, 0]

# Compute the power over time through the STFT.
t_stft, f_stft, x_stft = stft(x, win, hop_size)
x_psd = np.abs(x_stft)**2
x_pow = np.sum(np.abs(x_psd), axis=0)

### Ravel's Bolero: dynamic range

In [ ]:
plt.figure(figsize=(8, 3))

plt.plot(t_stft / fs / 60, 10 * np.log10(x_pow), color='blue')
plt.grid(True, which='both')
plt.xlabel('$t$ [min]')
plt.ylabel('$P(t)$ [dB]')
plt.title('Evolution of signal\'s power over time')

plt.show()

In [ ]:
# Boundaries of the excerpt we work with (in seconds).
t_start = 10
t_end = 27

# Parameters of the quantizer.
nbits = 10
nsteps = 2**nbits
midrise = True

# We assume that the signal amplitude is in interval [-1, 1]
step_size = 2 / nsteps

# Noise shaping filter.
a_h = np.array([1])
# Choose one of the filters listed in the cited paper.
# b_h = np.array([1])
# b_h = np.array([1.623, -0.982, 0.109])
b_h = np.array([2.412, -3.37, 3.937, -4.174, 3.353, -2.205, 1.281, -0.569, 0.0847]) 

x = data[int(t_start*fs):int(t_end*fs), 0]
t = np.arange(0, len(x)) / fs

nbits_is_6 = ipw.IntSlider(
    min=1, max=24, step=1, value=10, 
    description='$n_{\it bits}$', continuous_update=False)

midrise_cb_6 = ipw.Checkbox(description='midrise quantizer', value=True)

demo_out_6 = ipw.Output()

def update_quantize_bolero(nbits=10, midrise=True):
    nsteps = 2**nbits
    # We assume that the signal amplitude is in interval [-1, 1]
    step_size = 2 / nsteps
    
    # Quantize without dithering.
    y = quantize(x, step_size, midrise)

    # Generate dither.
    d_tpdf = dither_tpdf(step_size, len(x))

    # Quantize with dithering and noise shaping.
    y_ns = np.zeros(x.shape)
    if x.ndim > 1:
        for i in range(0, x.ndim):
            y_ns[:, i] = quantize_ns(x[:, i], d_tpdf, b_h, a_h, step_size, midrise)
    else:
        y_ns = quantize_ns(x, d_tpdf, b_h, a_h, step_size, midrise)
        
    audio_out_0 = ipw.Output()
    audio_out_1 = ipw.Output()
    audio_out_2 = ipw.Output()

    # Play the audio excerpts -- original and requantized.
    with audio_out_0:
        print('Original audio')
        display(ipd.Audio(data=np.clip(x, -1, 1), rate=fs, normalize=False))
    with audio_out_1:
        print('Quantized audio')
        display(ipd.Audio(data=np.clip(y, -1, 1), rate=fs, normalize=False))
    with audio_out_2:
        print('Quantized and noise-shaped audio')
        display(ipd.Audio(data=np.clip(y_ns, -1, 1), rate=fs, normalize=False))
    
    audio_group = ipw.HBox([audio_out_0, audio_out_1, audio_out_2])
    audio_group.layout = default_layout

    demo_out_6.clear_output()
    
    figure_out = ipw.Output()
    figure_out.layout = default_layout

    with figure_out:
        plt.figure(figsize=(15, 4))

        plt.subplot(1, 3, 1)
        plt.plot(t, x, color='blue')
        plt.xlabel('$t$ [s]')
        plt.ylabel('$s(t)$')
        plt.title('Original excerpt from Bolero', 
                  fontsize=12)
        plt.grid()

        plt.subplot(1, 3, 2)
        plt.plot(t, y, color='blue')
        plt.xlabel('$t$ [s]')
        plt.ylabel('$s(t)$')
        plt.title('Requantized w/o dither and noise shaping', 
                  fontsize=12)
        plt.grid()

        plt.subplot(1, 3, 3)
        plt.plot(t, y_ns, color='blue')
        plt.xlabel('$t$ [s]')
        plt.ylabel('$s(t)$')
        plt.title('Requantized w/ dithered noise shaping', 
                  fontsize=12)
        plt.grid()

        plt.tight_layout()
        plt.show()
        
    with demo_out_6:
        display(audio_group)
        display(figure_out)

def click_quantize_bolero(change):
    update_quantize_bolero(nbits=nbits_is_6.value, midrise=midrise_cb_6.value)

controls_6 = ipw.HBox([nbits_is_6, midrise_cb_6])
controls_6.layout = default_layout
run_button_6 = ipw.Button(description='Run')
run_button_6.on_click(click_quantize_bolero)

## Ravel's Bolero: distortion in the overture
- Example somewhat contrived since the recording appears properly quantized

In [ ]:
display(ipw.VBox([controls_6, run_button_6, demo_out_6]))
click_quantize_bolero(None)

# Dithering and noise shaping

- Other fields use similar ideas (*halftoning* and *error diffusion*)
- Can be combined with oversampling to achieve even higher SNR gains

**Note**: dithering is a much broader concept and is used in many other fields
- Example from previous life: distributed touch sensor design

In [ ]:
# Set the size of the scanned domain.
width = 1.5
length = 1.

# Set the spacing between emitters and detectors.
d_emitters = 0.25
d_detectors = 0.25

# Amplitude of the dither used to reduce the size of large holes.
dither_amp = 0.08

def dither(x, amp):
    return x + (np.random.rand(len(x)) - 0.5) * amp * 2

# Construct the arrays

# x positions at the bottom line.
x_emitters_bottom = np.arange(d_emitters / 2, width, d_emitters)
x_detectors_bottom = np.arange(d_emitters, width, d_detectors)
# x positions at the top line.
x_emitters_top = width - x_emitters_bottom
x_detectors_top = width - x_detectors_bottom

# y positions at the right line.
y_emitters_right = np.arange(d_emitters / 2, length, d_emitters)
y_detectors_right = np.arange(d_emitters, length, d_detectors)
# y positions at the left line.
y_emitters_left = length - y_emitters_right
y_detectors_left = length - y_detectors_right

# x positions along right and left lines.
x_emitters_right = width * np.ones(y_emitters_right.shape)
x_detectors_right = width * np.ones(y_detectors_right.shape)
x_emitters_left = np.zeros(y_emitters_left.shape)
x_detectors_left = np.zeros(y_detectors_left.shape)

# y positions along bottom and top lines.
y_emitters_bottom = np.zeros(x_emitters_bottom.shape)
y_detectors_bottom = np.zeros(x_detectors_bottom.shape)
y_emitters_top = length * np.ones(x_emitters_top.shape)
y_detectors_top = length * np.ones(x_detectors_top.shape)

# x and y positions of the entire emitter and detector arrays.
x_emitters = np.concatenate((x_emitters_bottom, x_emitters_right, 
                             x_emitters_top, x_emitters_left))
y_emitters = np.concatenate((y_emitters_bottom, y_emitters_right, 
                             y_emitters_top, y_emitters_left))
x_detectors = np.concatenate((x_detectors_bottom, x_detectors_right, 
                              x_detectors_top, x_detectors_left))
y_detectors = np.concatenate((y_detectors_bottom, y_detectors_right, 
                              y_detectors_top, y_detectors_left))

# Dither x positions at the top and bottom.
x_emitters_bottom_dithered = dither(x_emitters_bottom, dither_amp)
x_emitters_top_dithered = dither(x_emitters_top, dither_amp)
x_detectors_bottom_dithered = dither(x_detectors_bottom, dither_amp)
x_detectors_top_dithered = dither(x_detectors_top, dither_amp)

# Dither y positions on the left and the right side.
y_emitters_right_dithered = dither(y_emitters_right, dither_amp)
y_emitters_left_dithered = dither(y_emitters_left, dither_amp)
y_detectors_right_dithered = dither(y_detectors_right, dither_amp)
y_detectors_left_dithered = dither(y_detectors_left, dither_amp)

# x and y positions off the entire emitter and detector arrays.
x_emitters_dithered = np.concatenate((x_emitters_bottom_dithered, x_emitters_right, 
                                      x_emitters_top_dithered, x_emitters_left))
y_emitters_dithered = np.concatenate((y_emitters_bottom, y_emitters_right_dithered, 
                                      y_emitters_top, y_emitters_left_dithered))
x_detectors_dithered = np.concatenate((x_detectors_bottom_dithered, x_detectors_right, 
                                       x_detectors_top_dithered, x_detectors_left))
y_detectors_dithered = np.concatenate((y_detectors_bottom, y_detectors_right_dithered, 
                                       y_detectors_top, y_detectors_left_dithered))

emitter_pos = np.stack((x_emitters, y_emitters), axis=-1)
detector_pos = np.stack((x_detectors, y_detectors), axis=-1)

emitter_pos_dithered = np.stack((x_emitters_dithered, y_emitters_dithered), axis=-1)
detector_pos_dithered = np.stack((x_detectors_dithered, y_detectors_dithered), axis=-1)

def plot_arrays():
    # Plot the three projections.
    fig = plt.figure(figsize = (10, 5))

    plt.subplot(1, 2, 1)
    plt.scatter(emitter_pos[:, 0], emitter_pos[:, 1], color='b')
    plt.scatter(detector_pos[:, 0], detector_pos[:, 1], color='r')
    for i in range(emitter_pos.shape[0]):
        for j in range(detector_pos.shape[0]):
            plt.plot([emitter_pos[i, 0], detector_pos[j, 0]],
                    [emitter_pos[i, 1], detector_pos[j, 1]], color='g')
    plt.grid()
    plt.title("Regular array")
    plt.axis('scaled')

    plt.subplot(1, 2, 2)
    plt.scatter(emitter_pos_dithered[:, 0], emitter_pos_dithered[:, 1], color='b')
    plt.scatter(detector_pos_dithered[:, 0], detector_pos_dithered[:, 1], color='r')
    for i in range(emitter_pos_dithered.shape[0]):
        for j in range(detector_pos_dithered.shape[0]):
            plt.plot([emitter_pos_dithered[i, 0], detector_pos_dithered[j, 0]],
                    [emitter_pos_dithered[i, 1], detector_pos_dithered[j, 1]], color='g')
    plt.grid()
    plt.title("Dithered array")
    plt.axis('scaled')
    
    plt.show()

In [ ]:
plot_arrays()